In [55]:
import os

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [96]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from matplotlib import pyplot as plt

In [57]:
# setting the train and evaluation dataset directories. Change the below to your own path
train_path = '/content/drive/MyDrive/CS4243 Project/data/frames/train'
# root_path = '/content/drive/MyDrive/CS4243 Project/data/'
test_path = '/content/drive/MyDrive/CS4243 Project/data/frames/test'

In [103]:
image_size = (256,256)


train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_path,
    validation_split=0.2,
    subset="training",
    seed=110,
    image_size=image_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_path,
    validation_split=0.2,
    subset="validation",
    seed=110,
    image_size=image_size,
)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_path,
    seed=110,
)

Found 3537 files belonging to 2 classes.
Using 2830 files for training.
Found 3537 files belonging to 2 classes.
Using 707 files for validation.
Found 2071 files belonging to 2 classes.


In [104]:
image_size = (256,256)

data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.2),
        layers.experimental.preprocessing.Resizing(height=image_size[0], width=image_size[1]),
    ]
)
def make_model(input_shape):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    x = data_augmentation(inputs)

    # Rescale pixel values to the range [0,1]
    x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)

    # Single convolutional layer
    x = layers.Conv2D(16, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # x = layers.Conv2D(64, 3, strides=2, padding="same")(x)
    # x = layers.BatchNormalization()(x)
    # x = layers.Activation("relu")(x)
    # x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = layers.Flatten()(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dropout(0.5)(x)

    # Flatten layer
    x = layers.Flatten()(x)

    # Output layer
    outputs = layers.Dense(1, activation="sigmoid")(x)

    # Create the model
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [105]:
model = make_model(input_shape=image_size + (3,) )

In [106]:
# compiling and training our model

epochs = 2
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

In [107]:
history = model.fit(
    train_ds, epochs=epochs, validation_data=val_ds,
)

Epoch 1/2
89/89 [==============================] - 158s 2s/step - loss: 0.8440 - accuracy: 0.6099 - val_loss: 0.6847 - val_accuracy: 0.6563
Epoch 2/2
89/89 [==============================] - 165s 2s/step - loss: 0.6459 - accuracy: 0.5961 - val_loss: 0.6760 - val_accuracy: 0.5771


In [110]:
predictions = model.predict(test_ds)

65/65 [==============================] - 76s 1s/step


In [111]:
# Print the predicted class probabilities for the first sample in the validation dataset
print("Predictions for the first sample:")
# print(predictions[0])

# Assuming you have the ground truth labels for validation data (val_labels)
# Evaluate the model on the validation dataset
val_loss, val_accuracy = model.evaluate(test_ds)
print(f"Test Loss: {val_loss:.4f}")
print(f"Test Accuracy: {val_accuracy:.4f}")

Predictions for the first sample:
65/65 [==============================] - 78s 1s/step - loss: 0.6776 - accuracy: 0.6239
Test Loss: 0.6776
Test Accuracy: 0.6239


In [112]:
# Accuracy curve
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

AttributeError: ignored

In [ ]:
# Loss curve
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()